# Libraries

In [46]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta
import re

# Load and inspect dataset

In [47]:
# Get the current user's name
user = getuser()

In [48]:

# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\false_start\data\in\kate\2001_zurich.xlsx'

# Read the dataset with a different encoding
df = pd.read_excel(data_path)

display(df.head())

,2001 Weltklasse Zurich,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Result List 400m Hurdles Men B-Series 17.08...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,World Record,NaN,NaN,NaN,NaN,NaN,46.78,NaN,Kevin Young,NaN,...,NaN,Barcelone,NaN,NaN,NaN,NaN,NaN,1992-08-06,NaN,NaN
2,Meeting Record,NaN,NaN,NaN,NaN,NaN,47.10,NaN,Samuel Matete,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1991-08-07,NaN,NaN
3,rank,BIB,athlete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,finish time,NaN,NaN,NaN,NaN,NaT,NaN,reaction time
4,1,213,"GORBAN, Boris",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,48.58,NaN,NaN,NaN,NaN,NaT,NaN,0.262


In [49]:
def extract_table_info(df):
    # Define the columns we are interested in
    columns_of_interest = ['rank', 'BIB', 'athlete', 'nat', 'finish time', 'reaction time']
    
    # Initialize an empty list to store the tables
    tables = []
    metadata_list = []
    
    # Iterate through the DataFrame to find the start and end of each table
    start_indices = df[df.iloc[:, 0].str.contains('rank', na=False)].index
    end_indices = df[df.iloc[:, 0].str.contains('judge', na=False)].index
    
    for start, end in zip(start_indices, end_indices):
        # Extract metadata from the first three rows
        metadata_row = df.iloc[start-3:start].reset_index(drop=True)
        
        # Process first row (removing "Result List")
        first_row = metadata_row.iloc[0, 0].replace("Result List", "").strip()
        
        # Extract date from the last part of the string
        parts = first_row.rsplit(" ", 1)  # Split from the right
        event_info = parts[0].strip() if len(parts) > 1 else first_row.strip()
        date_str = parts[1].strip() if len(parts) > 1 else None
        
        # Convert date to proper format
        date = None
        if date_str:
            try:
                date = datetime.strptime(str(date_str), "%d.%m.%Y").date()  # Convert to string explicitly
            except ValueError:
                date = None
        
        # Extract detailed event info
        track_length = re.search(r"\b\d{2,4}m\b", event_info)  # Matches "400m", "100m", etc.
        track_length = track_length.group(0) if track_length else None

        hurdles = "Hurdles" if "Hurdles" in event_info else None
        gender = "Men" if "Men" in event_info else "Women" if "Women" in event_info else None
        
        # Remove extracted components from event_info to get category
        category = event_info
        if track_length:
            category = category.replace(track_length, "").strip()
        if hurdles:
            category = category.replace("Hurdles", "").strip()
        if gender:
            category = category.replace(gender, "").strip()
        category = category if category else None  # Avoid storing empty strings
        
        # Process second row (world record)
        wr_info = metadata_row.iloc[1].dropna().tolist()
        wr_athlete, wr_athlete_nat, wr_city, wr_date = (None, None, None, None)
        
        if len(wr_info) >= 4:
            wr_athlete = wr_info[2] if len(wr_info) > 2 else None
            wr_athlete_nat = wr_info[3] if len(wr_info) > 3 else None
            wr_city = wr_info[4] if len(wr_info) > 4 else None
            wr_date_str = wr_info[5] if len(wr_info) > 5 else None
            if wr_date_str:
                try:
                    wr_date = datetime.strptime(str(wr_date_str), "%d.%m.%Y").date()
                except ValueError:
                    wr_date = None
        
        # Process third row (meeting record)
        mr_info = metadata_row.iloc[2].dropna().tolist()
        mr_athlete, mr_athlete_nat, mr_city, mr_date = (None, None, None, None)
        
        if len(mr_info) >= 4:
            mr_athlete = mr_info[2] if len(mr_info) > 2 else None
            mr_athlete_nat = mr_info[3] if len(mr_info) > 3 else None
            mr_city = mr_info[4] if len(mr_info) > 4 else None
            mr_date_str = mr_info[5] if len(mr_info) > 5 else None
            if mr_date_str:
                try:
                    mr_date = datetime.strptime(str(mr_date_str), "%d.%m.%Y").date()
                except ValueError:
                    mr_date = None
        
        # Extract the table
        table = df.iloc[start+1:end].copy()
        
        # Set the first row as the header
        table.columns = df.iloc[start]
        
        # Clean column names by stripping whitespace
        table.columns = table.columns.str.strip()
        
        # Filter the columns of interest
        try:
            table = table[columns_of_interest]
        except KeyError as e:
            print(f"KeyError: {e}. Skipping this table.")
            continue
        
        # Drop rows with all NaN values
        table = table.dropna(how='all')
        
        # Extract judge and wind information from the row after the table
        judge_wind_info = df.iloc[end, 0]  # Assuming the judge/wind info is in the first column
        judge = None
        wind = None
        
        # Extract judge information
        if "judge:" in judge_wind_info:
            judge = judge_wind_info.split("judge:")[1].split("wind:")[0].strip()
            # Remove any content within parentheses
            judge = judge.split("(")[0].strip()
        
        # Extract wind information
        if "wind:" in judge_wind_info:
            wind = judge_wind_info.split("wind:")[1].strip()
        
        # Add extracted metadata to the table
        table['track_length'] = track_length
        table['hurdles'] = hurdles
        table['gender'] = gender
        table['category'] = category
        table['date'] = date
        table['judge'] = judge
        table['wind'] = wind
        
        # Add WR and MR details
        table['wr_athlete'] = wr_athlete
        table['wr_athlete_nat'] = wr_athlete_nat
        table['wr_city'] = wr_city
        table['wr_date'] = wr_date
        
        table['mr_athlete'] = mr_athlete
        table['mr_athlete_nat'] = mr_athlete_nat
        table['mr_city'] = mr_city
        table['mr_date'] = mr_date
        
        # Split athlete column into athlete_surname and athlete_name
        table['athlete_surname'] = table['athlete'].str.split(',').str[0].str.strip().str.lower()
        table['athlete_name'] = table['athlete'].str.split(',').str[1].str.strip().str.lower()
        
        # Drop the original athlete column
        table = table.drop(columns=['athlete'])
        
        # Append the table to the list
        tables.append(table)
        
        # Store metadata separately
        metadata_list.append({
            "track_length": track_length,
            "hurdles": hurdles,
            "gender": gender,
            "category": category,
            "date": date,
            "wr_athlete": wr_athlete,
            "wr_athlete_nat": wr_athlete_nat,
            "wr_city": wr_city,
            "wr_date": wr_date,
            "mr_athlete": mr_athlete,
            "mr_athlete_nat": mr_athlete_nat,
            "mr_city": mr_city,
            "mr_date": mr_date
        })
    
    return tables, metadata_list


In [50]:
# Extract the tables and metadata
tables, metadata_list = extract_table_info(df)

# Merge all tables into one final DataFrame
if tables:  # Ensure there is data before concatenating
    df_final = pd.concat(tables, ignore_index=True)
    
    # Print the final DataFrame
    print("Final DataFrame (df_final):")
    display(df_final.head(20))  # Show only the first 20 rows
else:
    print("No valid tables were extracted.")


KeyError: "['reaction time'] not in index". Skipping this table.
KeyError: "['finish time'] not in index". Skipping this table.
KeyError: "['finish time'] not in index". Skipping this table.
Final DataFrame (df_final):


C:\Users\aldi\AppData\Local\Temp\ipykernel_17824\1416313063.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat(tables, ignore_index=True)


,rank,BIB,nat,finish time,reaction time,track_length,hurdles,gender,category,date,...,wr_athlete,wr_athlete_nat,wr_city,wr_date,mr_athlete,mr_athlete_nat,mr_city,mr_date,athlete_surname,athlete_name
0,1,213,RUS,48.58,0.262,400m,Hurdles,Men,B-Series 17.08.2001,None,...,Kevin Young,USA,Barcelone,None,Samuel Matete,ZAM,1991-08-07,None,gorban,boris
1,2,218,POL,48.69,0.199,400m,Hurdles,Men,B-Series 17.08.2001,None,...,Kevin Young,USA,Barcelone,None,Samuel Matete,ZAM,1991-08-07,None,plawgo,marek
2,3,467,JAM,49.08,0.29,400m,Hurdles,Men,B-Series 17.08.2001,None,...,Kevin Young,USA,Barcelone,None,Samuel Matete,ZAM,1991-08-07,None,weakly,ian
3,4,217,CZE,49.09,0.312,400m,Hurdles,Men,B-Series 17.08.2001,None,...,Kevin Young,USA,Barcelone,None,Samuel Matete,ZAM,1991-08-07,None,muzik,jiri
4,5,215,RUS,49.13,0.262,400m,Hurdles,Men,B-Series 17.08.2001,None,...,Kevin Young,USA,Barcelone,None,Samuel Matete,ZAM,1991-08-07,None,machtchenko,rouslan
5,6,228,RSA,49.25,0.186,400m,Hurdles,Men,B-Series 17.08.2001,None,...,Kevin Young,USA,Barcelone,None,Samuel Matete,ZAM,1991-08-07,None,herbert,llewellyn
6,7,226,ITA,49.41,0.177,400m,Hurdles,Men,B-Series 17.08.2001,None,...,Kevin Young,USA,Barcelone,None,Samuel Matete,ZAM,1991-08-07,None,ottoz,laurent
7,8,229,FRA,49.42,0.175,400m,Hurdles,Men,B-Series 17.08.2001,None,...,Kevin Young,USA,Barcelone,None,Samuel Matete,ZAM,1991-08-07,None,theodore,olivier jean
8,1,260,NGR,11.06,0.17,100m,None,Women,B-Series 17.08.2001,None,...,Florence Griffith-Joyner,USA,Indianapolis,None,Evelyn Ashford,USA,1984-08-22,None,ojokolo,endurance
9,2,261,NGR,11.26,0.2,100m,None,Women,B-Series 17.08.2001,None,...,Florence Griffith-Joyner,USA,Indianapolis,None,Evelyn Ashford,USA,1984-08-22,None,onyali,mary
